<a href="https://colab.research.google.com/github/mssamoilenko/AnlDataPract/blob/main/AnlPrakt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практична робота №5

## Тема: Багаторукі бандити і дизайнер-новатор. Проведення тесту для довільної кількості варіантів.

## Мета роботи: закріплення навичок проведення тесту для довільної кількості варіантів із використанням алгоритму ϵ-жадібного бандиту, візуалізації та аналізу результатів тесту.

## Варіант №2

## Завдання

1. Завантажити дані індивідуального варіанту за посиланням на порталі
дистанційної освіти, використовуючи Pandas відобразити перші 5 рядків.

2. Запрограмувати алгоритм 𝜖-жадібного бандита. Встановити 𝜖 = 0,1.
3. При проведені тесту із 𝜖-жадібним бандитом всі дані не є доступними
відразу, натомість ви показуєте користувачам різні сторінки та збираєте
дані в режимі онлайн. Використовуючи запрограмований алгоритм 𝜖
жадібного бандита, зібрати як мінімум 300 прикладів. Приклади брати по
одному з даних за індивідуальним варіантом.
4. Показати:

a. Діаграму, де зобразити скільки разів було показано кожну із сторінок
і (на ній же) кількість відправлених листів для кожної із сторінок

b. Графік відношення відправлених листів до показів сторінки (для
кожної із сторінок) із часом. Для зображення використовувати
сукупні дані зібрані алгоритмом, за час взяти ітерацію.

5. Зазначити, який варіант є кращим.

    1. Згенерувати рівномірно розподілене випадкове число в інтервалі між 0 та 1.
    2. Якщо число знаходиться між 0 та 𝜖 (де 𝜖 (епсілон) — це число між 0 та 1 у типовій ситуації досить мале), випадково обрати один з варіантів іконки Gmail (№1, 2, 3, 4…).
    3. Якщо число більше або дорівнює 𝜖, то показати будь-яку пропозицію, яка дотепер мала найвище відношення відправлених листів.

## Хід роботи

1. Завантажую дані індивідуального варіанту за посиланням на порталі
дистанційної освіти, використовуючи Pandas відображаю перші 5 рядків.


In [12]:
import pandas as pd
import numpy as np

file_names = ['1.txt', '2.txt', '3.txt', '4.txt', '5.txt']

# Завантаження даних
data = [np.loadtxt(f) for f in file_names]
min_len = min(map(len, data))

# Створення DataFrame
clicks_df = pd.DataFrame(
    {str(i): data[i-1][:min_len] for i in range(1, 6)}
)

print(f"Розмірність даних: {clicks_df.shape}")
print(clicks_df.head())

Розмірність даних: (1000, 5)
     1    2    3    4    5
0  0.0  0.0  0.0  1.0  1.0
1  1.0  1.0  0.0  1.0  0.0
2  1.0  1.0  1.0  1.0  0.0
3  1.0  1.0  1.0  1.0  1.0
4  0.0  1.0  1.0  1.0  1.0


2. Програмую алгоритм 𝜖-жадібного бандита. Встановлюю 𝜖 = 0,1.